In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('Clothing-Reviews.csv')

In [3]:
df.head()

,asin,title,review_userId,review_score,review_summary,review_text
0,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A174NPQZ1EABX1,5,"Fundamental of death metal, do not miss","A roaring onslaught of streaming sound, this a..."
1,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A17ZI8VKZJFOV8,5,Great Classic,One of incantations best recordings. Im a fan ...
2,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2F9LXG1QEJ855,4,Very Original Debut of one of the most striden...,"This is an early, and rough-sounding look at t..."
3,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2FGXHSHF0OD17,5,No secret here,Whats to say about this album? Quintessential ...
4,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2O2E8BLB7VW1U,5,Brings a smile to my face...,...Whenever i listing to an album by incantati...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50046 entries, 0 to 50045
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   asin            50046 non-null  object
 1   title           50046 non-null  object
 2   review_userId   50046 non-null  object
 3   review_score    50046 non-null  int64 
 4   review_summary  47154 non-null  object
 5   review_text     50046 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.3+ MB


In [5]:
#df =df.drop(['review_summary', 'review_text',], axis=1)

In [6]:
df.head()

,asin,title,review_userId,review_score,review_summary,review_text
0,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A174NPQZ1EABX1,5,"Fundamental of death metal, do not miss","A roaring onslaught of streaming sound, this a..."
1,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A17ZI8VKZJFOV8,5,Great Classic,One of incantations best recordings. Im a fan ...
2,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2F9LXG1QEJ855,4,Very Original Debut of one of the most striden...,"This is an early, and rough-sounding look at t..."
3,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2FGXHSHF0OD17,5,No secret here,Whats to say about this album? Quintessential ...
4,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2O2E8BLB7VW1U,5,Brings a smile to my face...,...Whenever i listing to an album by incantati...


In [7]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['review_userId', 'asin', 'review_score']], reader)

In [8]:
from surprise import SVD
from surprise.model_selection import cross_validate
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.2705  1.2721  1.2607  1.2678  0.0051  
MAE (testset)     1.0053  1.0051  0.9965  1.0023  0.0041  
Fit time          2.56    2.48    2.61    2.55    0.06    
Test time         0.34    0.28    0.37    0.33    0.04    


{'test_rmse': array([1.27052474, 1.27212581, 1.26067118]),
 'test_mae': array([1.00530193, 1.00510259, 0.99648178]),
 'fit_time': (2.5574228763580322, 2.477360486984253, 2.6147236824035645),
 'test_time': (0.338686466217041, 0.27887988090515137, 0.3677024841308594)}

In [9]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [10]:
svd.predict(uid=10, iid=100)

Prediction(uid=10, iid=100, r_ui=None, est=4.121648083762938, details={'was_impossible': False})

In [11]:
import difflib
import random

def get_cloth_id(cloth_title, metadata):
    
    """
    Gets the cloth ID for a cloth title based on the closest match in the metadata dataframe.
    """
    
    existing_titles = list(metadata['title'].values)
    closest_titles = difflib.get_close_matches(cloth_title, existing_titles)
    cloth_id = metadata[metadata['title'] == closest_titles[0]]['asin'].values[0]
    return cloth_id

def get_cloth_info(cloth_id, metadata):
    
    """
    Returns some basic information about a cloth given the cloth id and the metadata dataframe.
    """
    
    cloth_info = metadata[metadata['asin'] == cloth_id][['review_userId','asin','title','review_summary']]
    return cloth_info.to_dict(orient='records')

def predict_review(user_id, cloth_title, model, metadata):
    
    """
    Predicts the review (on a scale of 1-5) that a user would assign to a specific cloth. 
    """
    
    cloth_id = get_cloth_id(cloth_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=cloth_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=3):
    
    """
    Generates a cloth recommendation for a user based on a rating threshold. Only
    cloths with a predicted rating at or above the threshold will be recommended
    """
    
    cloth_titles = list(metadata['title'].values)
    random.shuffle(cloth_titles)
    
    for cloth_title in cloth_titles:
        rating = predict_review(user_id, cloth_title, model, metadata)
        if rating >= thresh:
            cloth_id = get_cloth_id(cloth_title, metadata)
            return get_cloth_info(cloth_id, metadata)

In [12]:
generate_recommendation(200, svd, df)

[{'review_userId': 'A1JEJEDR7NDZ7C',
  'asin': 'B013LMKZRG',
  'title': 'Generic Womens Casual Crewneck Long Sleeve Cotton Long T-shirt Tops As picture S',
  'review_summary': 'I LOVE it!'}]